In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import healpy as hp

from holodeck import detstats, anisotropy
from holodeck.constants import YR

# Sato-Polito Process fow SWGB Modeling

1) Predict number of binaries in each bin in redshift, mass, and frequency

2) Obtain number of binaries for each realization in each bin by Poisson sampling

3) Uniformly sample angular positions (in $\cos \theta$ and $\phi$)

4) Get strain amplitude of each using eq. (7)
$$ h^2(z, \mathcal{M}, f) = \frac{32 \pi^{4/3}}{5 c^8} \frac{(1+z)^{10/3}}{d^2_L (z)} (\mathcal{G} \mathcal{M})^{10/3} f^{4/3} $$

5) Plug strain amplitude into Eq. (17)

$$ C_\ell (f) = \delta_{\ell 0}\delta_{m0} \bigg( \frac{f}{4\pi \Delta f}   \int d \vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h^2 (f,\vec{\theta})   \bigg)^2 
+ \big( \frac{f}{4 \pi \Delta f}\big)^2 \int d\vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h^4 (f, \vec{\theta})
$$


## Applying to our methods


$\frac{d N_{\Delta f}}{d \vec{\theta}}$ is the number in that frequency bin, at that angle

$ h_c^2 = \frac{f}{df} h_s^2 $ so this is the same as
$$ C_\ell (f) = \delta_{\ell 0}\delta_{m0} \bigg( \frac{1}{4\pi}   \int d \vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h_c^2 (f,\vec{\theta})   \bigg)^2 
+ \bigg( \frac{1}{4 \pi}\bigg)^2 \int d\vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h_c^4 (f, \vec{\theta} )
$$

$$ C_\ell (f) = \delta_{\ell 0}\delta_{m0} \bigg( \frac{1}{4\pi}   \int d \vec{\theta} h_c^2 (f)   \bigg)^2 
+ \bigg( \frac{1}{4 \pi} \bigg)^2 \int d\vec{\theta} h_c^4 (f )
$$



# Set Up
### Read in Strain Data

In [ ]:
sspath = '/Users/emigardiner/GWs/holodeck/output/2023-05-16-mbp-ss19_uniform05A_n1000_r50_d20_f30_l2000_p0/'
hdfname = sspath+'ss_lib.hdf5'
ssfile = h5py.File(hdfname, 'r')
print(list(ssfile.keys()))
hc_ss = ssfile['hc_ss'][...]
hc_bg = ssfile['hc_bg'][...]
fobs = ssfile['fobs'][:]
dfobs = ssfile['dfobs'][:]
ssfile.close()

shape = hc_ss.shape
nsamps, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
print('N,F,R,L =', nsamps, nfreqs, nreals, nloudest)


### Get best sample

In [ ]:
hc_ref15_10yr = 11.2*10**-15 
nsort, fidx, hc_tt, hc_ref15 = detstats.rank_samples(hc_ss, hc_bg, fobs, hc_ref=hc_ref15_10yr, ret_all=True)
print(hc_ref15)

### Get healpix map

In [ ]:
nside=32
moll_hc = anisotropy.healpix_map(hc_ss[nsort[0]], hc_bg[nsort[0]], nside=nside)

In [ ]:
rr=0
hp.mollview(moll_hc[rr,fidx], title='Sample %d, Realization %d, $f$=%.2f yr$^{-1}$' % (nsort[0], rr, fobs[fidx]*YR))

# Calculate Anisotropy

$$ C_\ell (f) = \delta_{\ell 0}\delta_{m0} \bigg( \frac{1}{4\pi}   \int d \vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h_c^2 (f,\vec{\theta})   \bigg)^2 
+ \bigg( \frac{1}{4 \pi}\bigg)^2 \int d\vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h_c^4 (f, \vec{\theta} )
$$

$$ C_\ell (f) = \delta_{\ell 0}\delta_{m0} \bigg( \frac{1}{4\pi}   \int d \vec{\theta} h_c^2 (f)   \bigg)^2 
+ \bigg( \frac{1}{4 \pi} \bigg)^2 \int d\vec{\theta} h_c^4 (f )
$$

Need to convert $ d\vec{\theta}$ to $d (\mathrm{pixel})$

In [ ]:
def spk_Cl(moll_hc):
    """ 
    
    """